In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Reshape

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [18]:
path = 'C:/Users/59482/Desktop/Columbia/Second Term/ads/Project3/Spring2020-Project3-ads-spring2020-project3-group3/data/'
dat_train = pd.read_csv(path+"dat_train.csv")
dat_test=pd.read_csv(path+"dat_test.csv")
dat_full=pd.read_csv(path+"dat_full.csv")
dat_full.drop('Unnamed: 0', inplace=True, axis=1)
dat_full

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature5998,feature5999,feature6000,feature6001,feature6002,feature6003,feature6004,feature6005,feature6006,emotion_idx
0,39,19,2,21,36,16,5,24,148,114,...,96,146,196,50,100,150,50,100,50,13
1,30,17,5,12,30,15,2,11,143,114,...,94,151,203,49,106,158,57,109,52,9
2,30,14,1,18,35,19,2,15,135,106,...,97,144,184,46,93,133,47,87,40,6
3,34,20,3,15,32,17,0,17,141,109,...,103,155,208,53,105,158,52,105,53,20
4,44,24,2,20,40,19,2,23,164,122,...,103,158,212,54,109,163,55,109,54,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,40,22,2,17,30,15,5,23,140,106,...,95,145,194,49,99,148,50,99,49,22
2496,38,19,3,13,36,20,2,20,160,119,...,109,162,208,53,106,152,53,99,46,22
2497,33,21,1,19,40,23,2,18,149,108,...,90,138,188,46,94,144,48,98,50,22
2498,35,18,3,20,36,19,0,21,153,114,...,93,144,197,49,100,153,51,104,53,22


In [113]:
X_train=dat_train.iloc[:,:-1]
Y_train=dat_train.iloc[:,-1]

X_test=dat_test.iloc[:,:-1]
Y_test=dat_test.iloc[:,-1]

from sklearn.decomposition import PCA
pca = PCA(n_components=0.99, whiten=True)
X_train_PCA = pca.fit_transform(X_train)
X_train_PCA

array([[-0.3859101 ,  0.42894874, -0.15423943, ..., -0.32021489,
         1.0694245 ,  0.41341669],
       [-0.64381387, -1.14091168, -0.15044509, ..., -1.61463967,
        -2.71085251, -0.48615836],
       [-1.60667719, -1.37835656,  0.79087003, ...,  1.25159942,
        -1.81300267,  2.12878709],
       ...,
       [ 1.45348759, -0.41980384, -0.72160376, ...,  0.0103879 ,
         1.59941309, -0.2952564 ],
       [-0.00503608, -0.32499587, -1.27477844, ...,  1.01079509,
         0.37585835, -0.4418095 ],
       [ 1.26017818,  0.74884621,  0.35056416, ...,  0.33231985,
         0.98482388, -1.19987908]])

In [53]:
len(X_train_PCA[1])

90

In [54]:
X_test_PCA = pca.transform(X_test)
X_test_PCA

array([[ 1.53867102,  0.29023548,  0.27804837, ...,  0.65310612,
        -0.35707205, -0.00646688],
       [ 0.73391199, -0.31705034,  1.1553738 , ..., -0.59843637,
        -1.16381385,  0.38111103],
       [ 0.58271613, -0.31543925,  1.07140477, ...,  1.49959578,
         1.30442532,  1.26280335],
       ...,
       [-0.64984464,  0.56098929, -0.19341404, ...,  1.26425914,
        -0.20173876,  0.75145747],
       [-0.28863336, -1.1846612 , -0.10493293, ..., -1.16047134,
        -0.27463411, -0.67990864],
       [-0.34002069,  1.82417531,  2.74760097, ...,  0.53858538,
        -2.13547731,  2.7507197 ]])

In [55]:
len(X_test_PCA[1])

90

In [56]:
#transform x
X_train_CNN = X_train_PCA.reshape(2000, 90,1)
X_test_CNN = X_test_PCA.reshape(500, 90,1)
#normalize x
X_train_CNN = X_train_CNN - np.mean(X_train_CNN, axis=0)
X_train_CNN = X_train_CNN/np.std(X_train_CNN, axis=0)

num_classes = 22

In [65]:
from keras.models import Sequential
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D,Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from keras import regularizers
from keras.regularizers import l2

model = Sequential()
#first layer is Convolution Layer
model.add(Conv1D(100, (3), input_shape=(90,1), activation='relu', kernel_regularizer=l2(0.1)))
#Second layer is Convolution Layer
model.add(Conv1D(100, (3), input_shape=(90,1), activation='relu'))
model.add(Dropout(0.2))
#Third layer is Convolution Layer
model.add(Conv1D(100, (3), input_shape=(90,1), activation='relu'))
model.add(Dropout(0.2))
#Fourth layer is Flatten Layer
model.add(Flatten())
model.add(Dropout(0.2))
#Fifth layer is Dense Layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
#Sixth layer is Dense Layer
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 88, 100)           400       
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 86, 100)           30100     
_________________________________________________________________
dropout_10 (Dropout)         (None, 86, 100)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 84, 100)           30100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 84, 100)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8400)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 8400)             

In [120]:
Y_train_binary = keras.utils.to_categorical(Y_train-1, num_classes)
Y_test_binary = keras.utils.to_categorical(Y_test-1, num_classes)
model.fit(X_train_CNN, Y_train_binary,
          epochs=20,
          verbose=1,
         validation_data=(X_test_CNN, Y_test_binary))

Train on 2000 samples, validate on 500 samples
Epoch 1/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1101 - accuracy: 0.9675 - val_loss: 3.1570 - val_accuracy: 0.4860
Epoch 2/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0814 - accuracy: 0.9745 - val_loss: 3.1082 - val_accuracy: 0.4740
Epoch 3/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0712 - accuracy: 0.9815 - val_loss: 3.1811 - val_accuracy: 0.4640
Epoch 4/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0686 - accuracy: 0.9850 - val_loss: 3.1081 - val_accuracy: 0.4760
Epoch 5/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0620 - accuracy: 0.9875 - val_loss: 3.0733 - val_accuracy: 0.4840
Epoch 6/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0448 - accuracy: 0.9910 - val_loss: 3.4816 - val_accuracy: 0.4900
Epoch 7/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0570 - accu